In [1]:
from glob import glob

In [52]:
r = glob('rationales/FT3/batch_*')
print(len(r))

14171


In [58]:
path = r[0]
path[path.find('_')+1:]

'13747'

In [6]:
print(len(glob('outputs/FT3/attentions/batch_*.pt')))

14345


In [2]:
2

2

In [3]:
from datasets import load_from_disk
batch = load_from_disk('kr4/FT3/batch_9')

In [4]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
def tokenize_func(example):
    return tokenizer(example['Rationale'], truncation=True)

In [5]:
# tokenize
batch = batch.map(tokenize_func, batched=True)

# process columns
batch = batch.remove_columns(['Unrationale', 'Rationale'])
batch = batch.rename_column('Rating', 'labels')
batch.set_format(type='torch')

Loading cached processed dataset at kr4/FT3/batch_9/cache-36e05c2708720c3f.arrow


In [9]:
from glob import glob
paths = glob(f'kr4/FT3/batch_*')

In [13]:
paths[0][paths[0].find('_')+1:]

'13747'

In [14]:
tokenized = load_from_disk('kr4_tokenized/FT3/batch_0')

In [15]:
tokenized

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 32
})

## PyTorch

In [22]:
from torch.utils.data import Dataset, DataLoader
from glob import glob

In [19]:
class RationaleDataset(Dataset):
    def __init__(self, rationale_path):
        self.rationale_path = rationale_path

    def __len__(self):
        return len(glob(self.rationale_path + '/batch_*'))

    def __getitem__(self, i):
        batch = load_from_disk(self.rationale_path + f'/batch_{i}')
        


In [20]:
dset = RationaleDataset('rationales/FT3/')

In [21]:
dset[1]

Dataset({
    features: ['Rating', 'Rationale', 'Unrationale'],
    num_rows: 32
})

In [23]:
dloader = DataLoader(dset)